## Долгосрочная работа по алгебре на тему алгоритм Берлекэмпа-Месси над конечными коммутативными кольцами с единицей. Часть 1.

## Срок сдачи 20.10.2022

Данная первая часть долгосрочной работы является несложной. Главная цель овладение системой КА Sage. Несколько советов.
- Пишите красивый и аккуратный код
- Разделяйте зону ответственности методов класса. Лучше написать 2-3 небольших метода класса чем один большой
- **Выводите отладочную информацию в файл**
- Поддерживайте порядок в jupyter-блокноте ~~это еще никому не удавалось~~

Необходимым условием для сдачи работы является написание недостающих участков кода, таким образом, чтобы прошли все тесты, и ответы на вопросы. 

In [3]:
import typing as tp # для подсказки типов

Реализуем класс отрезка последовательности над кольцом. 

In [4]:
class SequenceOverRing:
    def __init__(self, terms:tp.List, ring):
        self.terms = terms
        self.ring = ring
        self.ring_zero = self.ring.zero()
        self.period = None
    
    def __len__(self):
        return len(self.terms)
    
    def len(self):
        return len(self)
    
    def cnt_leading_zeros(self):
        ans = 0
        for i in range(len(self)):
            if self[i] == self.ring_zero:
                ans += 1
            else:
                break
        return ans
        
    def __getitem__(self, idx):
        return self.terms[idx]
    
    def __rmul__(self, pol:Polynomial):
        old_len = len(self)
        new_len = old_len - pol.degree()
        if new_len <= 0:
            return SequenceOverRing([], self.ring)
        new_terms = []
        for i in range(new_len):
            res = self.ring_zero
            for j in range(pol.degree()+1):
                 res = res + pol[j]*self.terms[i+j]
            new_terms.append(res)
        return SequenceOverRing(new_terms, self.ring)
    
    def __add__(self, other):
        new_len = min(self.len(), other.len())
        terms = []
        for i in range(new_len):
            terms.append(self[i] + other[i])
        return SequenceOverRing(terms, self.ring)
    
    
    def __neg__(self):
        return SequenceOverRing([-el for el in self.terms], self.ring)
    
    def is_empty(self):
        return len(self) == 0
    
    def is_zero(self):
        for term in self.terms:
            if term != self.ring_zero:
                return False
        return True
    
    def __sub__(self, other):
        terms = []
        for i in range(min(len(self), len(other))):
            terms[i] = self.terms[i] - other.terms[i]
        return SequenceOverRing(terms, self.ring)

    def check_is_period(self, t):
        ans = False
        n = len(self)
        for i in range(n - t):
            if self[i] != self[i+t]:
                return ans
        return True
        
    def check_is_minimal_period(self, t, t_proper_divs):
        if not self.check_is_period(t):
            return False
        ans = False
        for d in t_proper_divs:
            if self.check_is_period(d):
                return ans
        return True
             
    
    def compute_period(self, lam = 0):
        """
        TODO: make for non zero lambda
        """
        n = len(self)
        ans = -1
        for per in range(1, len(self)):
            # print(f'try {per}')
            was_break = False
            for i in range(n - per):
                if self[i] != self[i+per]:
                    was_break = True
                    break
            if not was_break:
                ans = per
                self.period = per
                return ans

Продемонстрируем работу класса. Создадим отрезок импульсной последовальнсти над $GF(2)$

In [7]:
R = GF(2)
terms = [R(0), R(0), R(1)]
u = SequenceOverRing(terms, ring = R)

К элементам последовательности можно обращаться по индексу

In [10]:
print(u[2])

1


При этом заметим, что элементы последовательности суть принадлежат полю из $2$ элементов.

In [11]:
print(type(u[2]))

<class 'sage.rings.finite_rings.integer_mod.IntegerMod_int'>


Также в классе реализованы вспомогательные функции типа длины отрезка и числа лидирующих нулей. Отрезки можно складывать и умножать на многочлен. Например.

In [20]:
R = IntegerModRing(6)
terms = [R(1), R(1), R(1)]

u = SequenceOverRing(terms, R)

Rx.<x> = PolynomialRing(R)
x = Rx.gen()
pol = x - R(1)
v = pol * u
print(f'u = {u.terms}, f ={ pol}, pol*u = {v.terms}')

u = [1, 1, 1], f =x + 5, pol*u = [0, 0]


**Вопрос**: как вы думаете, зачем при реализации класса последовательности мы явно передаем кольцо?

In [ ]:
## YOU ANSWER

Далее напишем класс ЛРП над кольцом. Данный класс является наследником класса `SequenceOverRing`. Это означает, что можно использовать все написанные выше функции для обычных последовательностей

In [21]:
class LRSoverRing(SequenceOverRing):
    def __init__(self, ring, init_vector, char_pol):
        super(LRSoverRing, self).__init__(terms = [el for el in init_vector], ring = ring)
        assert char_pol.is_monic(), 'characteristic polynomial should be monic'
        assert len(init_vector) == char_pol.degree()
        self.init_vector = init_vector
        self.char_pol = char_pol

    def compute_n_terms(self, n):
        for i in range(n):
            res = self.ring_zero
            for j in range(self.char_pol.degree()):
                res = res -self.char_pol[j] * self.terms[i+j]
            self.terms.append(res)


Продемонстрируем работу с классом. Создадим ЛРП максимального периода над $GF(2)$ порядка $3$.

In [29]:
R = GF(2)
Rx.<x> = PolynomialRing(R)
x = Rx.gen()

init_vec = [R(0), R(0), R(1)]
char_pol = x^3 - x - 1

u = LRSoverRing(R, init_vec, char_pol)
print(f'u = {u.terms}')

u = u.terms


Отметим, что мы можем пользоваться функциями описанными для класса последовательностей, например

In [25]:
print(len(u))

3


Далее вычислим 20 знаков ЛРП u

In [30]:
u.compute_n_terms(20)
print(f'u = {u.terms}')

u = [0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0]


Посчитаем период

In [32]:
per = u.compute_period()
print(per)

7


Рассмотрим другую ЛРП

In [35]:
init_vec = [R(0), R(0), R(1)]
char_pol = x^3 - x^2 - 1

v = LRSoverRing(R, init_vec, char_pol)
v.compute_n_terms(20)
print(f'v = {v.terms}')

v = [0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0]


Рассмотрим последовательность

In [42]:
w = [u[i*3 % 7] for i in range(23)]
w

[0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0]

**Вопрос**. Как связаны $w$ и $v$. Как вы можете это объяснить?

### Алгоритм Берлекэмпа-Месси из книги АЗКЧ

Вам нужно заполнить необходимые участки кода. Внимательно разберитесь с кодом. Для отладки алгоритма используйте параметр `verbose = True`.

**Совет** При выполнении задания можете писать отладочную информацию. Также можете выводить историю работы алгоритма в отдельный файл. Это очень упростит жизнь. В итоговой реализации никакой отладочной информации быть не должно.


In [45]:
class BerlecampMasseyOverField:
    def __init__(self, init_seq, verbose=False):
        self.init_seq = init_seq
        self.pol_ring = PolynomialRing(init_seq.ring, 'x')
        self.hist = []
        self.verbose = verbose
    
    def is_finished(self):
        l = self.hist[-1]['k'] + self.hist[-1]['m'] 
        if l == len(self.init_seq):
            return True
        
    def update_inf(self, g):
        m = g.degree()
        u = g * self.init_seq
        k = u.cnt_leading_zeros()
        self.hist.append({'g': g, 'k': k, 'm': m, 'u': u})
    
    
    def zero_step(self):
        x = self.pol_ring.gen()
        g0 = x^0
        
        self.update_inf(g0)
        return self.is_finished()

        
    def first_step(self):
        x = self.pol_ring.gen()
        
        k0 = self.hist[-1]['k']
        u0 = self.hist[-1]['u']
        g0 = self.hist[-1]['g']
        
        g1 = x^(k0 + 1) - u0[k0 + 1] * (u0[k0]^(-1)) * x^k0 * g0
        
        self.update_inf(g1)
        return self.is_finished()
        
    def step(self, idx_step):
        t = idx_step - 1
        s = t
        while self.hist[t]['m'] == self.hist[s]['m']:
            s -= 1      
        x = self.pol_ring.gen()
        
        ks = self.hist[s]['k']
        kt = self.hist[t]['k']
        
        gs = self.hist[s]['g']
        gt = self.hist[t]['g']
        
        us = self.hist[s]['u']
        ut = self.hist[t]['u']
        
        if kt <= ks:
            pass
            ### YOUR CODE HERE
        else:
            pass
            ### YOUR CODE HERE
            
        
        self.update_inf(g)
        return self.is_finished()
        
    def make_step(self, idx_step):
        if self.verbose:
            print(f'Step number {idx_step}')
            
        if idx_step == 0:
            return self.zero_step()
        if idx_step == 1:
            return self.first_step()
        else:
            return self.step(idx_step)
    
    def print_step_info(self, idx_step):
        print(f'After step idx {idx_step}:')
        for el in ['g', 'k', 'm']:
            print(f'{el}_{idx_step}: {self.hist[idx_step][el]}')
        print(f'u_{idx_step}: {self.hist[idx_step]["u"].terms}')
        print()

        
    def fit(self, verbose=True):
        is_finished = False
        idx_step = 0
        while not is_finished:
            is_finished = self.make_step(idx_step)
            if self.verbose:
                self.print_step_info(idx_step)
            idx_step += 1
        print(f'{self.hist[-1]["g"]}')

Приведенные тесты ниже должны проходить.

In [48]:
def test_bm_empty():
    print('-'*100)
    print('test bm empty')
    
    R = GF(3)
    seq = SequenceOverRing([], R)
    bm = BerlecampMasseyOverField(seq)
    bm.fit()
    print('-'*100)
    
    
def test_bm_simple():
    print('-'*100)
    print('test bm simple')
    R = GF(3)
    seq = SequenceOverRing([R(1)]*10 + [R.zero() for i in range(100)], R)
    bm = BerlecampMasseyOverField(seq)
    bm.fit()
    print('-'*100)

    
def test_bm_lrs():
    print('-'*100)
    print('test bm lrs')
    R = GF(2)
    S.<x> = PolynomialRing(R)
    seq = SequenceOverRing([R(0)]*4 + [R(1)], R)
    coefs = [-R(i) for i in range(1,6)]
    for i in range(1000):
        res = R(0)
        for j in range(5):
            res += coefs[j] * seq[i+j]
        seq.terms.append(res)
        
    
    bm = BerlecampMasseyOverField(seq)
    bm.fit()
    print('-'*100)
    
def test_bm_lrs_random():
    print('-'*100)
    print('test bm lrs')
    R = GF(49)
    S.<x> = PolynomialRing(R)
    
    degree = 13
    char_pol = S.random_element(degree=degree)
    char_pol = char_pol * (char_pol[char_pol.degree()]^(-1))
    #print(f'True char pol {char_pol}')
    seq = LRSoverRing(ring=R, init_vector=[R(0)]*(degree - 1) + [R(1)], char_pol=char_pol)
    seq.compute_n_terms(14)
    
    bm = BerlecampMasseyOverField(seq, verbose=False)
    bm.fit()
    print(bm.hist[-1]['g'] == char_pol)
    #print(f'Find char pol {bm.hist[-1]["g"]}')
    print('-'*100)

    
class RingAutomorplism:
    def __init__(self, ring):
        self.ring = ring
    def __call__(self, ring_elem):
        raise NotImplementedError
        
class GaloisRingAutomorphism(RingAutomorplism):
    def __init__(self, galois_ring, galois_ring_invariant):
        super(GaloisRingAutomorphism, self).__init__(galois_ring)
        self.galois_ring_invariant = galois_ring_invariant
        
    def __call__(self, ring_elem):
        raise NotImplementedError
        
class FiniteFieldAutomorphism(GaloisRingAutomorphism):
    def __init__(self, finite_field, finite_field_invariant):
        super(FiniteFieldAutomorphism, self).__init__(finite_field, finite_field_invariant)
    def __call__(self, ring_elem):
        return ring_elem^(self.galois_ring_invariant.cardinality())
        
            
class SkewLRS(SequenceOverRing):
    def __init__(self, ring, init_vec, char_pol_table: tp.List[tp.List], sigma):
        super(SkewLRS, self).__init__(terms=init_vec, ring=ring)
        self.sigma = sigma
        self.ring_ivariant = self.sigma.galois_ring_invariant
        self.char_pol_table = char_pol_table
    
    def show_char_pol_table(self):
        for row in self.char_pol_table:
            for el in row:
                print(el, end=' ')
            print()
    
    
    def compute_n_terms(self, n_terms):
        m = len(self.char_pol_table)
        for i in range(n_terms):
            res = self.ring_zero
            for j in range(m):
                sigma_pow = self[i+j]
                n = len(self.char_pol_table[j])
                for k in range(n):
                    res = res - self.char_pol_table[j][k]*sigma_pow
                    sigma_pow = self.sigma(sigma_pow)
            self.terms.append(res)

    
def test_bm_skew_lrs():
    print('-'*100)
    print('test bm  skew_lrs')
    R = GF(3)
    S = GF(9)
    m = 3
    init_vec = [S(0)] * (m-1) + [S(1)]
    char_pol_table = [
        [S(1), S(0)],
        [S([0,2]), S([0])],
        [S([2,1]), S([0])]
    ]
    print(char_pol_table)
    sigma = FiniteFieldAutomorphism(S, R)
    v =  SkewLRS(S, init_vec, char_pol_table, sigma)
    v.compute_n_terms(1000)
    
    print(v.terms)
    v.compute_period()
    print(v.period)
    
    bm = BerlecampMasseyOverField(v, verbose=True)
    bm.fit()
    print(bm.hist[-1]['g'])
    #print(f'Find char pol {bm.hist[-1]["g"]}')
    print('-'*100)
    

    
def test_bm():
    test_bm_empty()
    test_bm_simple()
    test_bm_lrs()
    for i in range(100):
        test_bm_lrs_random()
    test_bm_skew_lrs()
    
test_bm()
    

**Вопрос**. Приведите тест который сломает вашу реализацию.

**Подсказка**. Внимательно посмотрите на первый шаг. Насколько большим может быть ранг ЛРП.

In [ ]:
## YOU ANSWER

**Задание** 
При помощи алгоритма Берлекэмпа-Месси найдите все многочлены максимального периода порядка 4 над $GF(3)$ следующими способами.
- Рассмотрите импульсные последовательности с различными ХМ и заполните необходимое число знаков, после чего вычислите период. Если  ЛРП есть ЛРП МП, то  примените алгоритм БМ. Проверьте, что количество таких многочленов совпадает с теоретическим ответом.

- Постройте один МП многочлен. Далее пользуясь задачей о выборках из ЛРП постройте выборки с нужным шагом и примените алгоритм Берлекэмпа-Месси.

In [ ]:
## YOU ANSWER

Указанного недостатка, связанного с невозможностью применения описанного выше алгоритма к некоторым классам последовательностей лишен алгоритм БМ, рассказанный на ПЗ и прекрасно описанный в пособии В.Л. Куракина

## Алгоритм Берлекэмпа-Месси из пособия В.Л.Куракина

Внимательно разберитесь с кодом и заполните необходимые участки

Дадим некоторые пояснения. Ниже в алгоритме переменная для сохранения работы истории алгоритма используется переменная `hist`. Результаты **шагов** мы записываем в переменную `hist_step`. 

In [ ]:
class BerlecampMasseyVLOverField:
    def __init__(self, init_seq, verbose=False):
        self.init_seq = init_seq
        self.pol_ring = PolynomialRing(init_seq.ring, 'x')
        
        self.hist = []
        self.hist_step = []
        
        self.verbose = verbose
         
    
    def is_finished(self):
        l = self.hist[-1]['k'] + self.hist[-1]['m'] 
        if l == len(self.init_seq):
            return True
        
    def update_inf(self, g):
        m = g.degree()
        u = g * self.init_seq
        k = u.cnt_leading_zeros()
        
        self.hist.append({'g': g, 'k': k, 'm': m, 'u': u})
        
    def update_inf_step(self):
        self.hist_step.append(self.hist[-1])
        
        
    def zero_step(self):
        x = self.pol_ring.gen()

        g0 = x^0
        self.update_inf(g0)
        self.update_inf_step()
       
        return self.is_finished()
                    
    def is_kt_exist(self):
        t = None
        ks = self.hist[-1]['k']
        for i, el in enumerate(self.hist_step):
            if el['k'] == ks:
                t = i
                break
        return  t
        
    
    def print_step_info(self, idx_step):
        print(f'After step idx {idx_step}:')
        for el in ['g', 'k', 'm']:
            print(f'{el}_{idx_step}: {self.hist_step[idx_step][el]}')
        print(f'u_{idx_step}: {self.hist_step[idx_step]["u"].terms}')
        print()
    
    
    def step(self, idx):
        x = self.pol_ring.gen()
        
        g = self.hist_step[-1]['g'] * x
        self.update_inf(g)
        
        
        if self.is_finished():
            self.update_inf_step()
            return True
        
        t = self.is_kt_exist()
        while t is not None:
            u = self.hist[-1]['u']
            k = self.hist[-1]['k']
            
            # g = YOUR CODE HERE
            
            self.update_inf(g)
            t = self.is_kt_exist()
           
        self.update_inf_step()
        return self.is_finished()
            
        
            
    def make_step(self, idx_step):
        if self.verbose:
            print(f'Step number {idx_step}')
        if idx_step == 0:
            return self.zero_step()
        else:
            return self.step(idx_step)
    
    def fit(self, verbose=True):
        is_finished = False
        idx_step = 0
        while not is_finished:
            is_finished = self.make_step(idx_step)
            if self.verbose:
                self.print_step_info(idx_step)
            idx_step += 1
        #if self.verbose:
        print(f'{self.hist[-1]["g"]}')
              

Указанные ниже тесты должны проходить

In [64]:
def test_bm_empty():
    print('-'*100)
    print('test bm empty')
    
    R = GF(3)
    seq = SequenceOverRing([], R)
    bm = BerlecampMasseyVLOverField(seq)
    bm.fit()
    print('-'*100)
    
    
def test_bm_simple():
    print('-'*100)
    print('test bm simple')
    R = GF(3)
    seq = SequenceOverRing([R(1)]*10 + [R.zero() for i in range(10)], R)
    bm = BerlecampMasseyVLOverField(seq)
    bm.fit()
    print('-'*100)

def test_bm_full_rank():
    print('-'*100)
    print('test bm full rank')
    R = GF(3)
    seq = SequenceOverRing([R(0)]*10 + [R(1)], R)
    #seq.terms = seq.terms * 2
    bm = BerlecampMasseyVLOverField(seq)
    bm.fit()
    print('-'*100)
    
def test_bm_lrs():
    print('-'*100)
    print('test bm lrs')
    R = GF(2)
    S.<x> = PolynomialRing(R)
    seq = SequenceOverRing([R(0)]*4 + [R(1)], R)
    coefs = [-R(i) for i in range(1,6)]
    for i in range(1000):
        res = R(0)
        for j in range(5):
            res += coefs[j] * seq[i+j]
        seq.terms.append(res)
        
    
    bm = BerlecampMasseyVLOverField(seq)
    bm.fit()
    print('-'*100)
    
def test_bm_lrs_random():
    print('-'*100)
    print('test bm lrs')
    R = GF(49)
    S.<x> = PolynomialRing(R)
    
    degree = 13
    char_pol = S.random_element(degree=degree)
    char_pol = char_pol * (char_pol[char_pol.degree()]^(-1))
    #print(f'True char pol {char_pol}')
    seq = LRSoverRing(ring=R, init_vector=[R(0)]*(degree - 1) + [R(1)], char_pol=char_pol)
    seq.compute_n_terms(14)
    
    bm = BerlecampMasseyVLOverField(seq, verbose=False)
    bm.fit()
    print(bm.hist[-1]['g'] == char_pol)
    #print(f'Find char pol {bm.hist[-1]["g"]}')
    print('-'*100)


    
def test_bm_skew_lrs():
    print('-'*100)
    print('test bm  skew_lrs')
    R = GF(3)
    S = GF(9)
    m = 3
    init_vec = [S(0)] * (m-1) + [S(1)]
    char_pol_table = [
        [S(1), S(0)],
        [S([0,2]), S([0])],
        [S([2,1]), S([0])]
    ]
    print(char_pol_table)
    sigma = FiniteFieldAutomorphism(S, R)
    v =  SkewLRS(S, init_vec, char_pol_table, sigma)
    v.compute_n_terms(1000)
    
    print(v.terms)
    v.compute_period()
    print(v.period)
    
    bm = BerlecampMasseyVLOverField(v)
    bm.fit()
    print(bm.hist[-1]['g'])
    #print(f'Find char pol {bm.hist[-1]["g"]}')
    print('-'*100)
    

    
def test_bm():
    test_bm_empty()
    test_bm_simple()
    test_bm_lrs()
    test_bm_full_rank()
    for i in range(100):
        test_bm_lrs_random()
    test_bm_skew_lrs()
    
#test_bm()
    

### Нахождение минимального многочлена над некоторыми кольцами вычетов

На паре была указана возможность нахождения минимального многочлена для ЛРП над кольцами вычетов $Z_n$, где
$$
n = p_1\cdot p_2 \cdot \ldots \cdot p_k
$$

Напишите реализацию алгоритма Берлекэмпа-Месси для последовательностей над такими кольцами 

In [ ]:
### YOUR CODE HERE

Напишите тесты для своей реализации

In [49]:
### YOUR CODE HERE